<a href="https://colab.research.google.com/github/ericqu/ericqu.github.io/blob/master/fusedunfused.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#for issue 40510
!pip install tf-nightly
!pip install tensorflow-model-optimization


In [16]:
import gc
import sys
from datetime import date, datetime, time

import matplotlib.pyplot as plt
import numpy as np
import scipy as sci
import scipy.io as sio
import seaborn as sns
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import metrics
from tensorflow.keras.layers import (GRU, LSTM, Activation, Dense, Input,
                                     SimpleRNN)
from tensorflow.keras.models import Model
import tensorflow_model_optimization as tfmot

sns.set()
print(sys.version)
print("Tensor Flow:",  tf.__version__)
print("Keras: ", keras.__version__)
print("Numpy: ", np.__version__)
print("Seaborn: ", sns.__version__)
print("scipy: ", sci.__version__)


def increment_and_get(counter_path="counter.txt", increment=True):
    import os
    value = 0
    if os.path.exists(counter_path):
        f = open(counter_path, "r")
        value = int(f.readline())
        f.close()
        if increment is True:
            value = value + 1
    else:
        if increment is True:
            value = 1
        else:
            value = 0

    if increment is True:
        f = open(counter_path, "w")
        f.write(str(value))
        f.close()

    return value


model_iteration = increment_and_get(increment=False)


def get_files_desc(path="nodefault", cases=[0, 1, 2, 3, 4, 5, 6, 7]):
    Cases_description = ["Baseline", "Noise addition", "Adding respiratory movements for both mother and foetus", "Foetal movements added (helixoidal)",
                         "Maternal and foetal similar heart rate (alternatively changes in the heart rates)",
                         "Simulation of uterine contraction with noise and physiological based heart rate changes",
                         "Ectopic beats", "Twin pregnancy"
                         ]

    each_paths, each_descriptions = list(), list()
    for i in cases:
        full_path = path + str(i+1) + ".mat"
        each_paths.append(full_path)
        description = Cases_description[i]
        each_descriptions.append(description)
    return each_paths, each_descriptions


# cfg_cases_list = [2, 3, 4, 5]
cfg_cases_list = [1]
cfg_usable_cases_list = range(len(cfg_cases_list))
print ("Usable Cases list:", *cfg_usable_cases_list)


# cfg_EL_list = [5, 6, 7, 10, 11, 12, 15, 16, 17, 20, 21, 22, 25, 26, 27, 30, 31, 32 ] # Electrodes we are going to use
cfg_EL_list = range(3,34,5) # Electrodes we are going to use (from, to, step)
cfg_usable_EL_list = range(len(cfg_cases_list)) 
print ("Electrodes list:", *cfg_EL_list)

file_paths, Cases_descriptions = get_files_desc(
    "fecgsyn_c", cfg_cases_list)
print("file_paths:", file_paths)
print("Cases_descriptions", Cases_descriptions)

# configuration variables
cfg_mixture_gain = 100.0 
cfg_mother_gain = 100.0  
cfg_foetus_gain = 10.0   
cfg_in_len_seq = 250  # 350, 250, 150
cfg_out_len_seq = 50
cfg_batches_size = 500 # 5000 is too high - 4000 works at about same speed as 3000 ; 50 is 10 times slower than3000
cfg_epochs = 1
cfg_obs_ps = 500  # hertz
cfg_storage_type_f = np.float16

models = list()
models.append(dict(name="baseline_" + str(model_iteration), nb_neurons=20,
                   files=file_paths, per_file_description=Cases_descriptions, electrode_list= cfg_EL_list))

tf.random.set_seed(12345)
print("Model iteration: ", model_iteration)


3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
Tensor Flow: 2.3.0-dev20200618
Keras:  2.4.0
Numpy:  1.18.5
Seaborn:  0.10.1
scipy:  1.4.1
Usable Cases list: 0
Electrodes list: 3 8 13 18 23 28 33
file_paths: ['fecgsyn_c2.mat']
Cases_descriptions ['Noise addition']
Model iteration:  1


In [17]:
def load_mat(path):
    c_mat = sio.loadmat(path, variable_names=['out'])
    mat_out_data = c_mat['out']
    mat_out_type = mat_out_data.dtype
    c_data = {n: mat_out_data[n][0, 0] for n in mat_out_type.names}
    print("loading", path, " columns: ", str(
        [n for n, v in c_data.items()]), " EL, observations: ", c_data["mecg"].shape)

    mecg = np.array((c_data['mecg'] / cfg_mother_gain), dtype=cfg_storage_type_f)

    fecg1 = np.array(c_data["fecg"][0][0] / cfg_foetus_gain, dtype=cfg_storage_type_f)
    fecg2 = np.zeros(fecg1.shape, dtype=np.float32)
    fecg3 = np.zeros(fecg1.shape, dtype=np.float32)
    fecg4 = np.zeros(fecg1.shape, dtype=np.float32)

    mixture = np.array(c_data["mixture"] / cfg_mixture_gain, dtype=cfg_storage_type_f)

    # we have indexes, all is zero but the indexes
    mqrs_i = np.array(c_data["mqrs"], np.int64)
    mqrs = np.zeros(fecg1.shape[1], dtype=np.int8) # not per electrodes
    mqrs[mqrs_i] = 1;

    fqrs_i = np.array(c_data["fqrs"][0][0] , np.int64)
    fqrs = np.zeros(fecg1.shape[1], dtype=np.int8) # not per electrodes
    fqrs[fqrs_i] = 1;

    return mecg, fecg1, fecg2, fecg3, fecg4, mixture, mqrs, fqrs

all_mecg, all_fecg1, all_fecg2, all_fecg3, all_fecg4, all_mixture , all_mqrs , all_fqrs  = list(),list(), list(), list(), list(), list(), list(), list()

for c_path in file_paths:
    c_mecg, c_fecg1, c_fecg2, c_fecg3, c_fecg4, c_mixture, c_mqrs, c_fqrs = load_mat(c_path)
    print("completed.")

    all_mecg.append(c_mecg)
    all_fecg1.append(c_fecg1)
    all_fecg2.append(c_fecg2)
    all_fecg3.append(c_fecg3)
    all_fecg4.append(c_fecg4)
    all_mixture.append(c_mixture)
    all_mqrs.append(c_mqrs)
    all_fqrs.append(c_fqrs)

all_mecg = np.array(all_mecg)
all_fecg1 = np.array(all_fecg1)
all_fecg2 = np.array(all_fecg2)
all_fecg3 = np.array(all_fecg3)
all_fecg4 = np.array(all_fecg4)
all_mixture = np.array(all_mixture)
all_mqrs = np.array(all_mqrs)
all_fqrs = np.array(all_fqrs)


print("Total obs (all_mixture.shape):", all_mixture.shape)
print("Total obs (all_fqrs.shape):", all_fqrs.shape)

loading fecgsyn_c2.mat  columns:  ['mecg', 'mixture', 'mqrs', 'fqrs', 'fecg']  EL, observations:  (34, 20000)
completed.
Total obs (all_mixture.shape): (1, 34, 20000)
Total obs (all_fqrs.shape): (1, 20000)


In [18]:
def prep_electrodes_array(x, ym, yf, yqm, yqf, l_seq, o_seq, els, cases):
    gen_types = ["original", "no_foetus", "no_signal"]
    
    X, YM, YF1, YQM, YQF1 = None, None, None, None, None
    for case in cases:
        for el in els:
            for gen_type in gen_types:
                if X is None:  # First time
                    X, YM, YF1, YQM, YQF1= prep_training_array(x[case, el], ym[case,el], yf[case,el], yqm[case], yqf[case], l_seq, o_seq, gen_type)
                else:
                    X_t, YM_t, YF1_t, YQM_t, YQF1_t = prep_training_array(x[case, el], ym[case,el], yf[case,el], yqm[case], yqf[case], l_seq, o_seq, gen_type)
                    X = np.concatenate((X, X_t))
                    YM = np.concatenate((YM, YM_t))
                    YF1 = np.concatenate((YF1, YF1_t))
                    YQM = np.concatenate((YQM, YQM_t))
                    YQF1 = np.concatenate((YQF1, YQF1_t))

    return X, YM, YF1, YQM, YQF1


def prep_training_array(x, ym, yf, yqm, yqf, l_seq, o_seq, gen_type="original"):
    if (len(x) != len(ym)):
        raise NameError("x and y have different lengths " +
                        len(x) + " " + len(ym) + " cannot continue.")

    X = np.empty([len(x)-l_seq, l_seq, 1], dtype=cfg_storage_type_f)
    YM = np.empty([len(x)-l_seq, o_seq, 1], dtype=cfg_storage_type_f)
    YF1 = np.empty([len(x)-l_seq, o_seq, 1], dtype=cfg_storage_type_f)
    YQM = np.empty([len(x)-l_seq, o_seq, 1], dtype=np.int8)
    YQF1 = np.empty([len(x)-l_seq, o_seq, 1], dtype=np.int8)

    step = 1
    
    for i in range(0, len(x)-(l_seq), step):
        X[i] = x[i:i + l_seq].reshape(l_seq, 1)
        YM[i] = ym[i + l_seq - o_seq: i + l_seq].reshape(o_seq, 1)
        YF1[i] = yf[i + l_seq - o_seq: i + l_seq].reshape(o_seq, 1)
        YQM[i] = yqm[i + l_seq - o_seq: i + l_seq].reshape(o_seq, 1)
        YQF1[i] = yqf[i + l_seq - o_seq: i + l_seq].reshape(o_seq, 1)

    return X, YM, YF1, YQM, YQF1


print("Data Preparation started")
print("training data")
X_train, Y_trainM, Y_trainF1 , Y_trainQM, Y_trainQF1 = prep_electrodes_array(
    all_mixture, all_mecg, all_fecg1, all_mqrs, all_fqrs, cfg_in_len_seq , cfg_out_len_seq, cfg_EL_list, cfg_usable_cases_list)
print("test data")

X_test, Y_testM, Y_testF1, Y_testQM, Y_testQF1 = prep_training_array(
    all_mixture[0,8], all_mecg[0,8], all_fecg1[0,8], all_mqrs[0], all_fqrs[0], cfg_in_len_seq, cfg_out_len_seq)

print("shuffling")
together = np.hstack((X_train, Y_trainM, Y_trainF1 , Y_trainQM, Y_trainQF1))
np.random.shuffle(together)
X_train, Y_trainM, Y_trainF1, Y_trainQM, Y_trainQF1 = np.hsplit(together, [cfg_in_len_seq,
                                                                           cfg_in_len_seq+cfg_out_len_seq,
                                                                           cfg_in_len_seq+cfg_out_len_seq *2,
                                                                           cfg_in_len_seq+cfg_out_len_seq *3,])
def debug_display(name , val):
    print("debug:", name,  val.shape, val.dtype , "min :", np.min(val), "max:", np.max(val))

debug_display("X_train", X_train)
debug_display("Y_trainM", Y_trainM)
debug_display("Y_trainF1", Y_trainF1)
debug_display("Y_trainQM", Y_trainQM)
debug_display("Y_trainQF1", Y_trainQF1)

debug_display("X_test", X_test)
debug_display("Y_testM", Y_testM)
debug_display("Y_testF1", Y_testF1)
debug_display("Y_testQM", Y_testQM)
debug_display("Y_testQF1", Y_testQF1)

Data Preparation started
training data
test data
shuffling
debug: X_train (414750, 250, 1) float16 min : -283.8 max: 327.8
debug: Y_trainM (414750, 50, 1) float16 min : -271.0 max: 327.8
debug: Y_trainF1 (414750, 50, 1) float16 min : -1076.0 max: 431.5
debug: Y_trainQM (414750, 50, 1) float16 min : 0.0 max: 1.0
debug: Y_trainQF1 (414750, 50, 1) float16 min : 0.0 max: 1.0
debug: X_test (19750, 250, 1) float16 min : -90.2 max: 177.5
debug: Y_testM (19750, 50, 1) float16 min : -47.25 max: 140.1
debug: Y_testF1 (19750, 50, 1) float16 min : -119.5 max: 431.5
debug: Y_testQM (19750, 50, 1) int8 min : 0 max: 1
debug: Y_testQF1 (19750, 50, 1) int8 min : 0 max: 1


In [19]:
print(gc.collect())


1357


In [20]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Conv1D
from tensorflow.keras import regularizers
from tensorflow.keras import metrics


def define_model(length_of_sequences, batch_size=None, modelName="nonamegiven"):
    inp = Input(batch_shape=(batch_size, length_of_sequences, 1), name="inputs")
    lstmM = Bidirectional(LSTM(50, name="lstm_m", return_sequences=True))(inp)
    flat = Flatten()(lstmM)
    denseM = Dense(50, kernel_regularizer=regularizers.l2(0.0001))(flat)
    reshapeM = Reshape((50, 1))(denseM)
    denseM = TimeDistributed(
        Dense(1, kernel_regularizer=regularizers.l2(
            0.0001), bias_initializer='zeros'),
        input_shape=(50, 1))(reshapeM)
    out_M = Reshape((50, 1), name="om")(denseM)
    model = Model(inputs=[inp], outputs=[out_M], name=modelName)
    model.compile(
        loss={"om": "mean_squared_error"},
        optimizer=keras.optimizers.RMSprop(learning_rate=0.001),
        metrics=['mae'])
    
    return model

def define_q_model(k_model): 
    q_aware_model = tfmot.quantization.keras.quantize_model(k_model)
    q_aware_model.compile(
        loss={"quant_om": "mean_squared_error", "quant_of": "mean_squared_error"},
        optimizer=keras.optimizers.RMSprop(learning_rate=0.001), 
        metrics=['mae'])
    
    return q_aware_model
    
for m in models:
    m["k_model"] = define_model(
        length_of_sequences=cfg_in_len_seq, modelName=m["name"])
    m["k_model"].summary()
    


Model: "baseline_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 250, 1)]          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 250, 100)          20800     
_________________________________________________________________
flatten_1 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                1250050   
_________________________________________________________________
reshape_1 (Reshape)          (None, 50, 1)             0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 1)             2         
_________________________________________________________________
om (Reshape)                 (None, 50, 1)             0

In [ ]:
# Training

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=5, restore_best_weights= True)

for m in models:
    print ("fitting")
    m["k_history"] = m["k_model"].fit(x=X_train, y=[Y_trainM, Y_trainF1], 
                                      batch_size=cfg_batches_size, epochs=cfg_epochs, verbose=1, validation_split=0.1, 
                                      callbacks=[early_stop])

fitting
  1/747 [..............................] - ETA: 0s - loss: 1351.6111 - mae: 15.1670